In [1]:
from unidecode import unidecode


In [2]:


#1ero: ver que productos exporta la region de rio grande do sul 
#2do: quedarme las filas que tienen los productos de rio grande do sul, en la base expo provincial

import pandas as pd


expo_provincias_argentinas = pd.read_excel('C:\\Users\\dwaisman\\Desktop\\EXPROV19-23.xlsx', sheet_name='EXPOPROV1923', dtype={'POSIC':str})

expo_provincias_argentinas = expo_provincias_argentinas[expo_provincias_argentinas['AÑO']==2023]

expo_brasil = pd.read_csv('C:\\Users\\dwaisman\\Desktop\\base_mergeada_2023_2024_4.csv', dtype={'CO_NCM':str})


expo_brasil = expo_brasil[expo_brasil['CO_ANO']==2023]

filas_mantener = ['Rio Grande do Sul']


#rio_grande es la base de brasil con la que voy a trabajar


rio_grande = expo_brasil[expo_brasil.NO_UF.isin(filas_mantener)]



productos_rio_grande = rio_grande.groupby(['CO_NCM', 'NO_PAIS_ESP']).agg(Count=('CO_NCM', 'size'),Total_VL_FOB=('VL_FOB', 'sum')).reset_index()



lista_productos = rio_grande['CO_NCM'].unique().tolist()


lista_paises_expo_brasil = rio_grande['NO_PAIS_ESP'].unique().tolist()


lista_paises_expo_brasil = [unidecode(pais.lower()) for pais in lista_paises_expo_brasil]


In [3]:
lista_productos_argentina = expo_provincias_argentinas['POSIC'].unique().tolist()


lista_paises_expo_argentina = expo_provincias_argentinas['PAISDESTIN'].unique().tolist()


lista_paises_expo_argentina = [unidecode(pais.lower()) for pais in lista_paises_expo_argentina]


#filas_que_me_imp_de_exp_br = rio_grande[rio_grande.CO_NCM.isin(lista_productos_argentina)]

tabla_rio_grande = rio_grande.groupby(['CO_NCM', 'NO_PAIS_ESP']).agg(Count=('CO_NCM', 'size'),Total_VL_FOB=('VL_FOB', 'sum')).reset_index()

lista_paises_expo_prov_args = expo_provincias_argentinas['PAISDESTIN'].unique().tolist()

lista_paises_expo_prov_args = [unidecode(pais.lower()) for pais in lista_paises_expo_prov_args]

set_paises_expo_brasil = set(lista_paises_expo_brasil)
set_paises_expo_prov_args = set(lista_paises_expo_prov_args)

# Encontrar la intersección
interseccion = set_paises_expo_brasil.intersection(set_paises_expo_prov_args)

# Contar los elementos de la intersección
cantidad_coincidencias = len(interseccion)


diferencia_brasil = set_paises_expo_brasil.difference(set_paises_expo_prov_args)
diferencia_prov_args = set_paises_expo_prov_args.difference(set_paises_expo_brasil)


diferencia_brasil_list = list(diferencia_brasil)
diferencia_prov_args_list = list(diferencia_prov_args)

# Crear una lista combinada de elementos que no tienen intersección
sin_interseccion = list(diferencia_brasil.union(diferencia_prov_args))

In [4]:


import difflib



# Encontrar posibles coincidencias
posibles_coincidencias = {}
for pais in diferencia_prov_args_list:
    mejor_coincidencia = difflib.get_close_matches(pais, diferencia_brasil_list, n=1, cutoff=0.6)
    if mejor_coincidencia:
        posibles_coincidencias[pais] = mejor_coincidencia[0]




posibles_coincidencias.pop('islas salomon', None)
posibles_coincidencias.pop('corea, republica de', None)
posibles_coincidencias.pop('san martin (parte holandesa)', None)
posibles_coincidencias['camboya (ex kampuchea)'] = 'camboya'
posibles_coincidencias['corea, republica de'] = 'corea (sur)'

correciones = posibles_coincidencias


In [5]:
productos_por_prov = expo_provincias_argentinas.groupby(['POSIC', 'PAISDESTIN','DESCPROVI']).agg(Count=('POSIC', 'size'),Total_VL_FOB=('FOB', 'sum')).reset_index()

productos_por_prov['nuevo_PAISDESTIN'] = productos_por_prov['PAISDESTIN'].str.lower().apply(unidecode)

productos_por_prov['nuevo_PAISDESTIN'] = productos_por_prov['nuevo_PAISDESTIN'].replace(posibles_coincidencias)

productos_rio_grande['nuevo_PAISDESTIN'] = productos_rio_grande['NO_PAIS_ESP'].str.lower().apply(unidecode)

productos_por_prov['formerge'] = productos_por_prov.apply(lambda row: f"{row['POSIC']}-{row['nuevo_PAISDESTIN']}", axis=1)


productos_rio_grande['formerge'] = productos_rio_grande.apply(lambda row: f"{row['CO_NCM']}-{row['nuevo_PAISDESTIN']}", axis=1)



resultado = pd.merge(productos_rio_grande, productos_por_prov, on='formerge', how='left')




resultado.to_excel(r'C:\Users\dwaisman\OneDrive - FUNDACION ARGENTINA PARA LA PROMOCION DE INVERSIONES Y COMERCIO INTERNACIONAL\IC 2022\Interno\3. Pedidos especiales\cruceBrasilProvincia6.xlsx')



In [7]:
resultado

,CO_NCM,NO_PAIS_ESP,Count_x,Total_VL_FOB_x,nuevo_PAISDESTIN_x,formerge,POSIC,PAISDESTIN,DESCPROVI,Count_y,Total_VL_FOB_y,nuevo_PAISDESTIN_y
0,01012100,Argentina,10,651601,argentina,01012100-argentina,NaN,NaN,NaN,NaN,NaN,NaN
1,01012100,Paraguay,2,49829,paraguay,01012100-paraguay,01012100,Paraguay,Buenos Aires,1.0,8000.0,paraguay
2,01012100,Uruguay,12,348888,uruguay,01012100-uruguay,01012100,Uruguay,Buenos Aires,1.0,203050.0,uruguay
3,01012100,Uruguay,12,348888,uruguay,01012100-uruguay,01012100,Uruguay,Extranjero,1.0,2600.0,uruguay
4,01012900,Estados Unidos,1,70000,estados unidos,01012900-estados unidos,01012900,Estados Unidos,Buenos Aires,1.0,4585850.0,estados unidos
...,...,...,...,...,...,...,...,...,...,...,...,...
53843,97039000,Eslovenia,1,185,eslovenia,97039000-eslovenia,NaN,NaN,NaN,NaN,NaN,NaN
53844,97039000,Estados Unidos,9,14924,estados unidos,97039000-estados unidos,NaN,NaN,NaN,NaN,NaN,NaN
53845,97039000,Filipinas,1,260,filipinas,97039000-filipinas,NaN,NaN,NaN,NaN,NaN,NaN
53846,97039000,Francia,1,445,francia,97039000-francia,NaN,NaN,NaN,NaN,NaN,NaN
